In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.pyplot import figure
from main_func import main,real_partial
from plotting import plot
from jupyterthemes import jtplot

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:35195' processes=5 threads=5, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:35195 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=30000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.01 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":1000,
    "n_informative":10,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.5,0.5],
    "random_state":17
}
threshold=0.03

chunks=100
e_array=[0.01,0.1,0.6]
minibatches=[5,10,50]
new="yes" #choose if you want to recreate the dataset

### Dataset with no noise flip_y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="yes"
dataset_params["weights"]=[0.5,0.5]
Acc_threshold=[]
Rounds_threshold=[]
Time_threshold=[]
labels=[[]]
l1=[[]]
l2=[[]]

for i in e_array :
    e=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatches[0])
    new="no"
    Acc_threshold.append(Acc)
    Rounds_threshold.append(n_rounds)
    Time_threshold.append(time_stamps)
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])

labels[0]="Real"
print(len(Acc_threshold[0]),Rounds_threshold,len(Time_threshold[0]))
l1[0]=len_real1
l2[0]=Acc_real1
name="Plots/Threshold_w_"+len(w)+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_e_"+str(e_array)+"_a.png"
plot(l1,l2,Acc_threshold,Rounds_threshold,Time_threshold,e_array,labels,name)


Start with num of chunks: 100 and e: 0.01
A new dataset has been created...
Size of chunk:  300
Number of chunks for each worker:  25
In progress...


In [ ]:
# 3 different e
#new="yes"
dataset_params["weights"]=[0.5,0.5]
Acc_minibatch=[]
Rounds_minibatch=[]
Time_minibatch=[]
labels=[[]]
l1=[[]]
l2=[[]]
e=0.1
for i in minibatches :
    minibatche=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatch)
    new="no"
    Acc_minibatch.append(Acc)
    Rounds_minibatch.append(n_rounds)
    Time_minibatch.append(time_stamps)
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])

labels[0]="Real"
print(len(Acc_threshold[0]),Rounds_threshold,len(Time_threshold[0]))
l1[0]=len_real1
l2[0]=Acc_real1
name="Plots/Minibatch_w_"+len(w)+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_e_"+str(e_array)+"_a.png"
plot(l1,l2,Acc_minibatch,Rounds_minibatch,Time_minibatch,e_array,labels,name)


### Save results

In [ ]:
f = open("result.txt", "a")
f.writelines(["\n\n\nNEW EXPERIMENT:\nparams:",str(dataset_params),"\nminibaches_size:"
              ,str(minibaches),"\ndifferent e's:",str(e_array),"\nn_workers: ",str(len(w))])
f.writelines(["\n\nRESULTS:\nBal_rounds_e",str(Bal_rounds_e),"\nBal_acc_e",str(Bal_acc_e),
            "\nBal_rounds_b",str(Bal_rounds_b),"\nBal_acc_b",str(Bal_acc_b),"\nUnBal_rounds_e", 
              str(UnBal_rounds_e),"\nUnBal_acc_e",str(UnBal_acc_e),"\nUnBal_rounds_b",
              str(UnBal_rounds_b),"\nBal_acc_b",str(UnBal_acc_b),"\nbal_e_time",str(bal_e_time),
              "\nbal_b_time",str(bal_b_time),"\nunbal_e_time",str(unbal_e_time),
              "\nunbal_b_time",str(unbal_b_time),"\nlen_real1",str(len_real1),
              "\nAcc_real1",str(Acc_real1),"\nlen_real2",str(len_real2),
              "\nAcc_real2",str(Acc_real2)])
f.close()

### Shutdown client

In [ ]:
client.close()
client.shutdown()